# Initialize Kaggle API

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
AnalyzeBoston/crimes-in-boston                              Crimes in Boston                                     10MB  2018-09-04 17:56:03           9368  
matheusfreitag/gas-prices-in-brazil                         Gas Prices in Brazil                                  3MB  2019-07-12 16:23:52           5153  
jealousleopard/goodreadsbooks                               Goodreads-books                                     632KB  2019-06-14 16:35:05           8855  
wkirgsn/electric-motor-temperature                          Electric Motor Temperature                           47MB  2019-06-19 11:12:41           3346  
jolasa/waves-measuring-buoys-data-mooloolaba                Wave

In [ ]:
!kaggle competitions download -c ittodaydsc2019

  0% 0.00/11.0k [00:00<?, ?B/s]
100% 11.0k/11.0k [00:00<00:00, 19.8MB/s]
  0% 0.00/1.86M [00:00<?, ?B/s]
100% 1.86M/1.86M [00:00<00:00, 61.6MB/s]
  0% 0.00/61.7k [00:00<?, ?B/s]
100% 61.7k/61.7k [00:00<00:00, 50.7MB/s]


In [ ]:
!unzip QuadData.csv.zip
!rm QuadData.csv.zip

Archive:  QuadData.csv.zip
  inflating: QuadData.csv            


# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Submission Example

In [ ]:
import pandas as pd
example = pd.read_csv('sampleSubmission.csv')
example.head(10)

,id,Lng,Lat
0,516,106.733964,-6.557158
1,517,106.733964,-6.557158
2,518,106.733964,-6.557158
3,519,106.733964,-6.557158
4,520,106.733964,-6.557158
5,521,106.733964,-6.557158
6,522,106.733964,-6.557158
7,523,106.733964,-6.557158
8,524,106.733964,-6.557158
9,525,106.733964,-6.557158


# Files Description
Hazim sedang mengerjakan project kuliah yaitu merakit dua buah quadcopter. Quadcopter adalah helikopter multirotor yang diangkat dan didorong oleh empat rotor (baling-baling). Quadcopter yang dirakit Hazim diatur untuk terbang dengan dua tipe rute lintasan secara otomatis (autopilot) serta dapat dipantau melalui remote control ataupun komputer. Pada quadcopter tersebut terdapat beberapa sensor, diantaranya adalah global positioning system (GPS), global position accuracy (GPA), inertial measurement unit (IMU), dan magnetometer (Kompas). Setelah tahap perakitan selesai, Hazim akan melakukan uji coba penerbangan pada quadcopter rakitan nya untuk diambil data nya. Melalui kedua quadcopter tersebut, dilakukan uji coba terbang dengan mengikuti koordinat longitude dan latitude pada GPS quadcopter sesuai dengan rute yang telah diatur sebelumnya. Tugas anda yaitu prediksi data koordinat (longitude,latitude) deret waktu dari posisi ke-X hingga posisi terakhir untuk setiap percobaan.

Hasil akan dievaluasi menggunakan metric Mean Haversine Distance

QuadData.csv Data rekam hasil percobaan, untuk data dengan nilai koordinat (Longitude,Latitude) bernilai NaN akan menjadi data test yang harus diprediksi
description.xlsx Deskripsi dari setiap fitur pada dataset
sampleSubmission.csv Contoh submisi yang valid, file berisi nilai prediksi dari dataset QuadData yang memiliki nilai koordinat NaN

**Comments**
* From 8791 data points in QuadData.csv, 6440 are samples for training and 2351 are samples for testing. So we need to split it.
* description.xlsx is hard to read literally apparently.


# Read the Data

In [ ]:
from pandas_profiling import ProfileReport as Report
Data = pd.read_csv('QuadData.csv')
len(Data.columns)

93

## Step 1: Drop constant and exact columns

In [ ]:
immediate_drop = ['DSAlt', 'GWk', 'MOfsX', 'MOfsX_2', 'MOfsY', 'MOfsY_2', 'MOfsZ', 'MOfsZ_2', 'SAlt', 'SMS', 'VV']
Data = Data.drop(immediate_drop, axis = 1)
len(Data.columns)
Data.to_csv('QuadData_01.csv', index = False)
!cp QuadData_01.csv 'drive/My Drive/ITToday'

## Step 2: Drop highly correlated columns
with $\rho>0.9$.


In [ ]:
highcorr_drop = ['AccX_2', 'AccY_2', 'Alt', 'BAlt', 'C4', 'GyrX_2', 'GyrY_2', 'GyrZ_2',
                 'HDop', 'TimeUS_ATT', 'TimeUS_CTUN', 'TimeUS_GPA', 'TimeUS_GPS',
                 'TimeUS_IMU', 'TimeUS_IMU2', 'TimeUS_MAG', 'TimeUS_MAG2', 'TimeUS_RCOU',
                 'MagX_2', 'MagY_2', 'OfsX_2', 'OfsY_2', 'OfsZ_2', 'ThO']
highcorr_drop += ['LineNo_CTUN', 'LineNo_GPA', 'LineNo_GPS', 'LineNo_IMU', 'LineNo_IMU2',
                  'LineNo_MAG', 'LineNo_MAG2', 'LineNo_RCOU', 'OfsX', 'OfsY', 'OfsZ']
Data = Data.drop(highcorr_drop, axis = 1)
len(Data.columns)
Data.to_csv('QuadData_02.csv', index = False)
!cp QuadData_02.csv 'drive/My Drive/ITToday'

## Step 3: Split Train and Test Data

In [ ]:
to_drop = Data[Data.isnull().any(axis = 1)].index
to_drop = list(to_drop)
len(to_drop)

2351

In [ ]:
Data_test  = Data.iloc[to_drop].reset_index().drop('index', axis = 1)
Data_train = Data.drop(to_drop, axis = 0).reset_index().drop('index', axis = 1)

print('train shape:', Data_train.shape)
print('test shape :', Data_test.shape)

train shape: (6440, 47)
test shape : (2351, 47)


## Step 4: Outlier Removal
From analysis, we find some outliers that fit these conditions:
* If ExpID = 3 and LineNo_ATT <= 9158 or >= 14968
* If ExpID = 4 and LineNo_ATT <= 2076
* If ExpID = 10 and LineNo_ATT <= 8857

In [ ]:
outliers_index3  = (Data_train['ExpID'] == 3 ) & ((Data_train['LineNo_ATT'] <= 9158) | (Data_train['LineNo_ATT'] >= 14968))
outliers_index4  = (Data_train['ExpID'] == 4 ) &  (Data_train['LineNo_ATT'] <= 2076)
outliers_index10 = (Data_train['ExpID'] == 10) &  (Data_train['LineNo_ATT'] <= 8857)

print('number of outlier samples:', sum([len(Data_train[outlier_idx]) for outlier_idx in [outliers_index3, outliers_index4, outliers_index10]]))

number of outlier samples: 123


In [ ]:
outliers_index = outliers_index3 | outliers_index4 | outliers_index10
Data_Train = Data_train[~outliers_index].reset_index().drop('index', axis = 1)
print('new train shape:', Data_Train.shape)

new train shape: (6317, 47)


## Step 5: Test Data Value Set Analysis

In [ ]:
CatLen = []
for col in Data_test.columns:
  CatLen.append(len(set(Data_test[col])))
print('distribution of', len(CatLen), 'columns in test data:')
d = dict(zip(Data_test.columns, CatLen))
#print(d)
print({k: d[k] for k in ('ExpID', 'PatternID', 'DeviceID', 'ErrRP', 'ErrYaw', 'VDop',
                         'HAcc', 'VAcc', 'SAcc', 'Delta', 'Status', 'NSats')})

distribution of 47 columns in test data:
{'ExpID': 10, 'PatternID': 2, 'DeviceID': 2, 'ErrRP': 23, 'ErrYaw': 52, 'VDop': 24, 'HAcc': 25, 'VAcc': 46, 'SAcc': 36, 'Delta': 10, 'Status': 2, 'NSats': 10}


we find these columns to be categorical:
* ExpID: 10 unique values
* PatternID: 2 unique values
* DeviceID: 2 unique values
* ErrRP: 23 unique values
* ErrYaw: 52 unique values
* VDop: 24 unique values
* HAcc: 25 unique values
* VAcc: 46 unique values
* SAcc: 36 unique values
* Delta: 10 unique values
* Status: 2 unique values
* NSats: 10 unique values

Description of each categorical columns:
* ExpID: ID of Experiment
* PatternID: Route used in Experiment
* DeviceID: Quadcopter device (device 1 or device 2)
* ErrRP: The average size of the roll/pitch error estimate (values between 0 and 1)
* ErrYaw: The average size of the yaw error estimate (values between 0 and 1)
* VDop: Vertical dilution of precision, a unitless measure of precision
* HAcc: Horizontal Accuracy as reported by the GPS module, in meters
* VAcc: Vertical Accuracy as reported by the GPS module, in meters
* SAcc: Speed accuracy as reported by the GPS, in m/s/s
* Delta: The time between when the previous GPS message and the current GPS message was parsed by the autopilot, in milliseconds
* Status: 0 = no GPS, 1 = GPS but no fix, 2 = GPS with 2D fix, 3 = GPS with 3D fix
* NSats: The number of satellites current being used

In [ ]:
CatLen = []
for col in Data_Train.columns:
  CatLen.append(len(set(Data_Train[col])))
print('distribution of', len(CatLen), 'columns in train data:')
d = dict(zip(Data_Train.columns, CatLen))
#print(d)
print({k: d[k] for k in ('ExpID', 'PatternID', 'DeviceID', 'ErrRP', 'ErrYaw', 'VDop',
                         'HAcc', 'VAcc', 'SAcc', 'Delta', 'Status', 'NSats')})

distribution of 47 columns in train data:
{'ExpID': 12, 'PatternID': 2, 'DeviceID': 2, 'ErrRP': 21, 'ErrYaw': 74, 'VDop': 43, 'HAcc': 29, 'VAcc': 62, 'SAcc': 45, 'Delta': 16, 'Status': 2, 'NSats': 12}


In [ ]:
t_dist = {'ExpID': 10, 'PatternID': 2, 'DeviceID': 2, 'ErrRP': 23, 'ErrYaw': 52, 'VDop': 24, 'HAcc': 25, 'VAcc': 46, 'SAcc': 36, 'Delta': 10, 'Status': 2, 'NSats': 10}
T_dist = {'ExpID': 12, 'PatternID': 2, 'DeviceID': 2, 'ErrRP': 21, 'ErrYaw': 74, 'VDop': 43, 'HAcc': 29, 'VAcc': 62, 'SAcc': 45, 'Delta': 16, 'Status': 2, 'NSats': 12}

check = [col for col in ('ExpID', 'PatternID', 'DeviceID', 'ErrRP', 'ErrYaw', 'VDop',
                         'HAcc', 'VAcc', 'SAcc', 'Delta', 'Status', 'NSats') if T_dist[col] > t_dist[col]]
print(check)
for key in check:
  print(set(Data_Train[key]) - set(Data_test[key]))

['ExpID', 'ErrYaw', 'VDop', 'HAcc', 'VAcc', 'SAcc', 'Delta', 'NSats']
{3, 5}
{0.67, 0.5, 0.43, 0.34, 0.68, 0.35, 0.44, 0.7, 0.45, 0.37, 0.71, 0.54, 0.46, 0.38, 0.47, 0.72, 0.39, 0.56, 0.73, 0.4, 0.74, 0.41}
{0.95, 1.19, 1.29, 1.32, 1.35, 1.17, 1.03, 1.04, 1.33, 1.01, 1.51, 1.26, 1.09, 1.6, 1.02, 0.96, 1.05, 0.89, 0.98, 1.06, 1.15, 0.9}
{0.42, 0.7, 0.71, 0.72}
{0.68, 1.25, 1.27, 1.26, 1.34, 1.28, 1.33, 1.21, 1.29, 1.22, 1.3, 1.23, 1.31, 0.74, 1.24, 1.32}
{0.52, 0.53, 0.62, 0.45, 0.46, 0.38, 0.47, 0.55, 0.56, 0.4, 0.49}
{202, 203, 204, 185, 188, 189}
{17, 18}


Let's remove values by ExpID and NSats

In [ ]:
remove_ExpID = (Data_Train['ExpID'] == 3 ) | (Data_Train['ExpID'] == 5 )
remove_NSats = (Data_Train['NSats'] == 17) | (Data_Train['NSats'] == 18)

print('number of removed samples if ExpID filter applied:', len(Data_Train[remove_ExpID]))
print('number of removed samples if NSats filter applied:', len(Data_Train[remove_NSats]))
print('number of removed samples if both filters applied:', len(Data_Train[remove_ExpID]) + len(Data_Train[remove_NSats]))

number of removed samples if ExpID filter applied: 1201
number of removed samples if NSats filter applied: 483
number of removed samples if both filters applied: 1684


In [ ]:
remove_index = remove_ExpID | remove_NSats
Train_Data_v1 = Data_Train[~remove_ExpID].reset_index().drop('index', axis = 1)
Train_Data_v2 = Data_Train[~remove_NSats].reset_index().drop('index', axis = 1)
Train_Data_vb = Data_Train[~remove_index].reset_index().drop('index', axis = 1)
print('new train shape v1:', Train_Data_v1.shape)
print('new train shape v2:', Train_Data_v2.shape)
print('new train shape vb:', Train_Data_vb.shape)

new train shape v1: (5116, 47)
new train shape v2: (5834, 47)
new train shape vb: (4923, 47)


In [ ]:
Data_test.to_csv('test.csv', index = False)

Data_Train.to_csv('train.csv', index = False)
Train_Data_v1.to_csv('train_ExpID_filter.csv', index = False)
Train_Data_v2.to_csv('train_NSats_filter.csv', index = False)
Train_Data_vb.to_csv('train_both_filter.csv', index = False)

!cp *.csv 'drive/My Drive/ITToday'

# End of Pre-processing